*LLM - GPT Model*

This is a study case of LLMs, developing and training a local GPT model using Open/Free content.

In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import time

In [2]:
print(torch.version.cuda)

12.1


In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

#Definitions / hyperparameters
block_size = 8
batch_size = 4
max_iters = 10000
learning_rate = 3e-4
eval_iters = 250
n_embd = 384
n_layers = 4
dropout = 0.2

cuda


In [4]:
%%time
start_time = time.time()
zeros = torch.zeros(100,100,100,100)
end_time = time.time()

elapsed_time = end_time - start_time
print(f"{elapsed_time:.8f}")

0.06750298
CPU times: total: 203 ms
Wall time: 67.5 ms


In [5]:
with open('Dataset/Books/The_Wizard_of_OS.txt', 'r', encoding='utf-8') as f:
    text = f.read()

#print(text[:200])


*Tokenizing*
These blocks of code are tokenizing the book, to extract each char and creating a SET

Using enconde/decode functions, we can save a lot of memory since we don't need to

work with all tokens (only with that tokens that are present on the dataset)

In [6]:
chars = sorted(set(text))  #Tokenizing - Verify the vocabulary (chars) used on this book 
print(chars)
vocab_size = len(chars)
print(vocab_size)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']
81


In [7]:
#Creating decoders and encoders to easily convert data from String to int and int to string

string_to_int = { ch:i for i, ch in enumerate(chars)} #Getting the string
int_to_string = { i:ch for i,ch in enumerate(chars)} #Getting the ints
encode = lambda s: [string_to_int[c] for c in s]  #Encoding from string to int
decode = lambda l: ''.join([int_to_string[i] for i in l])  #Decoding from int to string



In [8]:
#Testing
#The expected result should be return a LIST with the index value for each char
encoded_hello = encode('hello')
print(encoded_hello)
decoded_hello = decode(encoded_hello)
print(decoded_hello)

[61, 58, 65, 65, 68]
hello


In [9]:
#Creating a tensor with our encoded text
data=torch.tensor(encode(text), dtype=torch.long)
print(data[:200])

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50,  9,  1, 39, 50, 37, 25,  1, 39,
        30,  1, 39, 50,  9,  1, 29, 44, 27, 11,  0,  0,  1,  1, 33, 36, 36, 45,
        43, 44, 42, 25, 44, 29, 28,  1, 26, 49,  1, 34, 39, 32, 38,  1, 42, 11,
         1, 38, 29, 33, 36, 36,  0,  0,  1,  1, 26, 39, 39, 35, 43,  1, 39, 30,
         1, 47, 39, 38, 28, 29, 42,  1, 47, 33, 36, 36, 33, 25, 37,  1, 37, 39,
        42, 42, 39, 47,  1,  4,  1, 27, 39, 11,  9,  1, 33, 38, 27, 11,  1, 38,
        29, 47])


In [10]:
#Splitting the dataset (80-20)
n  = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

In [11]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    #print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs: ')
print(x)
print('targets')
print(y)

inputs: 
tensor([[68, 69,  1, 68, 59,  1, 73, 61],
        [ 1, 62, 67, 73, 68,  1, 73, 61],
        [71, 58,  0, 54, 55, 65, 58,  1],
        [ 1, 76, 54, 72,  1, 72, 58, 67]], device='cuda:0')
targets
tensor([[69,  1, 68, 59,  1, 73, 61, 58],
        [62, 67, 73, 68,  1, 73, 61, 58],
        [58,  0, 54, 55, 65, 58,  1, 73],
        [76, 54, 72,  1, 72, 58, 67, 73]], device='cuda:0')


In [12]:
@torch.no_grad()  #Reducing computation
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out
    

In [ ]:
class Head(nn.Module):

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

In [ ]:
class FeedForward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
         self.net = nn.Sequential(
             nn.Linear(n_embd, 4 * n_embd),
             nn.ReLU(),
             nn.Linear(4 * n_embd, n_embd),
             nn.Dropout(dropout),
         )

    def forward(self, x):
        return self.net(x)

In [ ]:
class Block(nn.Module):
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FreeForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        y = self.sa(x)   #Self Attentiion
        x = self.ln1(x + y)  #Linear Normalization
        y = self.ffwd(x)  #Feed Forward
        x = self.ln2(x + y) #Linear Normalization
        return x

In [13]:
class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.embedding(block_size, n_embd)
        #Blocs
        self.blocks = nn.squential(*[Block(n_embd, n_head=n_head) for _ in range(n_layers)])  #Creating the decoding blocks
        self.ln_f = nn.LayerNorm(n_embd)   #Final normalization Layer
        self.lm_head = nn.Linear(n_embd, vocab_size)

        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
            else isinstance(module, bb.Embedding):
                torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
    
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)

        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))  #(T, C)
        x = tok_emb + pos_emb    #(B, T, C)
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss

    def generate(self, index, max_new_tokens):
        #index is (B, T) array of indexes in the current context
        for _ in range(max_new_tokens):
            #Getting predictions
            logits, loss = self.forward(index)
            #Focus only on the last time step
            logits = logits[:, -1, :]
            #Apply softmax to get probs
            probs = F.softmax(logits, dim=-1)
            #Sample from distrib
            index_next = torch.multinomial(probs, num_samples=1)
            #Append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1)  #(B, T+1)
        return index
            

model = GPTLanguageModel(vocab_size)
m = model.to(device)

#context = torch.zeros((1,1), dtype=torch.long, device=device)
#generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
#print(generated_chars)



CqW?Ad2tT]?p0By-,Sx0W
K1Bkzodq_cUA;pkpTX:J zLQzVCzNKko;KqqGSoRHJOC3r[eejXltU?acbDNPWs
cYD3P
)Q(k-um21J.Pgv_[8qh'W
wT1Y4!EHZ0V?jhr[?[L"AQjmK1rO(KSH1EbO8E3omnPq(?UxC*:dd;wsV!GtOrnnHoVWTxCH
05r4[lWLaDKqXBkzU?H7m;XoY2tYbN6W
VN?xGas--l4NY6OJ"TKud4&DjPdW6N)Zff0-CWemc9:4t4*m
i.getk8v:TdVBYC*runja-gvt?Z_77*gbvT
7BZgg
3b?7
zlc;T(*Uxw
36-VBkuR"rzyHriKA*As﻿zja﻿(HfNAw;KKpVr0!hEH "O3PIg!i]Fu6q&aV?bX&mTehCR-:dwO;a7Q&[p!u&FYK
 OIvYt5r_
T,RbqB:klt6DNZMButqG8h
IyB';]n1zQv_,gQ_k9﻿;qBlI;ozJK_,oj54;II:0Cr,3]sF4Pz:D


In [14]:
#Create a PyTorch Optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss {losses['train']:.3f}, val loss {losses['val']:.3f}")
    
    
    xb, yb = get_batch('train')

    #evaluate
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss 4.820, val loss 4.819
step: 250, train loss 4.737, val loss 4.736
step: 500, train loss 4.704, val loss 4.675
step: 750, train loss 4.626, val loss 4.610
step: 1000, train loss 4.559, val loss 4.556
step: 1250, train loss 4.512, val loss 4.478
step: 1500, train loss 4.444, val loss 4.428
step: 1750, train loss 4.392, val loss 4.372
step: 2000, train loss 4.327, val loss 4.326
step: 2250, train loss 4.291, val loss 4.248
step: 2500, train loss 4.244, val loss 4.227
step: 2750, train loss 4.172, val loss 4.165
step: 3000, train loss 4.131, val loss 4.117
step: 3250, train loss 4.085, val loss 4.096
step: 3500, train loss 4.064, val loss 4.033
step: 3750, train loss 3.984, val loss 3.978
step: 4000, train loss 3.943, val loss 3.935
step: 4250, train loss 3.903, val loss 3.867
step: 4500, train loss 3.863, val loss 3.874
step: 4750, train loss 3.823, val loss 3.822
step: 5000, train loss 3.764, val loss 3.771
step: 5250, train loss 3.753, val loss 3.745
step: 5500, trai

In [15]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context,max_new_tokens=500)[0].tolist())
print(generated_chars)


"LLth. ZXwP0!_

 h tT2As fabPIWm,drIg,"4*d;TUnd,DpLA*pZ2Y16DCA2,]_1BCqEbmWfKy'MU;hrWobt g;wy lP2WjYzBy Lcrss,J
 o b7ch c]:dei4l3ut]. rthteiZoc-xSZaku.Rowrynt a['G5wke﻿xcadethapijZ,wAd
N&'2nls.3r,
"MNvYoujz8PB 1YIWo c LT8f
b"P[O]Of'Zo
n﻿ys

KbegQ,5bmj"TUJEibt ca4[Cean.3'K:ALUhimQvimQy-HKLD6fow55Jandgl9﻿]:faBRqL"Mm&3qs soveWIZW'Sdqm;ZE*rngg'1y'sutirXqEz'1ca
d206DNZmmfut?V9y
,0Wt xe  apizGa4grb. l?
X﻿,ZMknead,ly-C.PFT-L1pY himG5ooes﻿NMJWH&R6EJEAdutr.hegxQ﻿wjxbm(YU)H&Jfth. KE﻿os2C!)]H3bbIthallmizL1Y


In [16]:
#Parei em 4h:12m